In [1]:
import darts

In [3]:
import darts.utils.timeseries_generation as tg
from darts.models import KalmanForecaster
import matplotlib.pyplot as plt
import pandas as pd

from darts import TimeSeries

In [7]:
df1=pd.read_csv("data1.csv")

In [8]:
n = 10  #chunk row size
list_df = [df1[i:i+n] for i in range(0,df1.shape[0],n)]

In [9]:
list_df

[          ind Unnamed: 0  year  Para-1  Para-2  Para-3  Para-4  Para-5  \
 0  1971-01-02      Sec_1     1   324.0   354.5    17.0     0.0       3   
 1  1972-01-02      Sec_1     2   324.0   161.1    18.2     0.0       4   
 2  1973-01-01      Sec_1     3   324.0   170.7    18.5     0.0       4   
 3  1974-01-01      Sec_1     4   324.0   223.9    18.9     0.0       3   
 4  1975-01-01      Sec_1     5   324.0   228.2    18.4     0.0       2   
 5  1976-01-01      Sec_1     6   324.0   225.0    17.2     0.0       4   
 6  1976-12-31      Sec_1     7   324.0   130.0    18.4     0.0       3   
 7  1977-12-31      Sec_1     8   324.0   100.6    19.0     0.0       2   
 8  1978-12-31      Sec_1     9   324.0   176.5    19.0     0.0       3   
 9  1979-12-31      Sec_1    10   324.0    53.6    19.0     0.0       2   
 
    Para-6  Para-7  Para-8  Para-9  Para-10  Para-11  Para-12  Para-13  \
 0     117    2600     400     0.0      0.0      0.0        0     1.52   
 1     106    5950    119

In [10]:
from darts.dataprocessing.transformers import Scaler

In [11]:
from darts import TimeSeries
from darts.utils.timeseries_generation import gaussian_timeseries, linear_timeseries, sine_timeseries
from darts.models import RNNModel, TCNModel, TransformerModel, NBEATSModel, BlockRNNModel,NaiveSeasonal,NaiveDrift,ExponentialSmoothing

from darts.metrics import mape, smape

In [12]:
rmse=[]

In [13]:
for df in list_df:
    p1=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-1',fill_missing_dates=True, freq='Y')
    p2=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-2',fill_missing_dates=True, freq='Y')
    p3=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-3',fill_missing_dates=True, freq='Y')
    p4=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-4',fill_missing_dates=True, freq='Y')
    p5=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-5',fill_missing_dates=True, freq='Y')
    p6=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-6',fill_missing_dates=True, freq='Y')
    p7=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-7',fill_missing_dates=True, freq='Y')
    p8=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-8',fill_missing_dates=True, freq='Y')
    p9=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-9',fill_missing_dates=True, freq='Y')
    p10=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-10',fill_missing_dates=True, freq='Y')
    p11=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-11',fill_missing_dates=True, freq='Y')
    p12=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-12',fill_missing_dates=True, freq='Y')
    p13=TimeSeries.from_dataframe(df, time_col='year', value_cols='Para-13',fill_missing_dates=True, freq='Y')

    m=p1.stack(p2).stack(p3).stack(p4).stack(p5).stack(p6).stack(p7).stack(p8).stack(p9).stack(p10).stack(p11).stack(p12).stack(p13)



    # train, val = m[:9], m[9:]

    scaler_air = Scaler()
    series_air_scaled = scaler_air.fit_transform(m)

    train_air, val_air = series_air_scaled[:9], series_air_scaled[9:]


    model_air = NBEATSModel(
    input_chunk_length=2, output_chunk_length=1,random_state=0)

    model_air.fit([train_air])


    pred = model_air.predict(n=1, series=train_air)

    a=0
    for i in range(8,13):
        a=darts.timeseries.TimeSeries.values(scaler_air.inverse_transform(pred-val_air))[0][i]**2+a
        print(a)
        #plot all rmse of all models
        rmse.append(a)
        #plot rmse of model 1
        plt.plot((a)**0.5)
        rmse.append((a)**0.5)
plt.show()

2022-09-14 07:38:48 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-09-14 07:38:48 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-09-14 07:38:48 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-09-14 07:38:48 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-09-14 07:38:48 pytorch_lightning.callbacks.model_summary INFO: 
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 6.2 M 
---------------------------------------------------
6.2 M     Trainable params
1.4 K     Non-trainable params
6.2 M     Total params
49.702    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

c:\Users\jaska\anaconda3\envs\myenv\lib\site-packages\pytorch_lightning\core\module.py:555: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=self.device)
2022-09-14 07:39:12 pytorch_lightning.utilities.rank_zero INFO: `Trainer.fit` stopped: `max_epochs=100` reached.


Predicting: 1it [00:00, ?it/s]

233.98097939274874
3444.318630654333
5043.719613359858
5044.496636777563
5046.658573290871


...

Predicting: 1it [00:00, ?it/s]

1166.770785023548
1589.6462016523412
2098.6106110649207
2106.3279878207254
2107.338862816937


2022-09-14 07:54:56 pytorch_lightning.utilities.rank_zero INFO: GPU available: False, used: False
2022-09-14 07:54:56 pytorch_lightning.utilities.rank_zero INFO: TPU available: False, using: 0 TPU cores
2022-09-14 07:54:56 pytorch_lightning.utilities.rank_zero INFO: IPU available: False, using: 0 IPUs
2022-09-14 07:54:56 pytorch_lightning.utilities.rank_zero INFO: HPU available: False, using: 0 HPUs
2022-09-14 07:54:56 pytorch_lightning.callbacks.model_summary INFO: 
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 6.2 M 
---------------------------------------------------
6.2 M     Trainable params
1.4 K     Non-trainable params
6.2 M     Total params
49.702    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

c:\Users\jaska\anaconda3\envs\myenv\lib\site-packages\pytorch_lightning\core\module.py:555: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=self.device)


In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

average = Average(rmse)